In [1]:
import pandas as pd

## Register Status

In [2]:
register_status = {}

# Registradores inteiros (x0..x31)
for i in range(32):
    register_status[f"x{i}"] = {"x": i, "x_type": "int", "writer": None}

# Registradores de ponto flutuante (f0..f31)
for i in range(32):
    register_status[f"f{i}"] = {"f": i, "f_type": "float", "writer": None}

register_status


{'x0': {'x': 0, 'x_type': 'int', 'writer': None},
 'x1': {'x': 1, 'x_type': 'int', 'writer': None},
 'x2': {'x': 2, 'x_type': 'int', 'writer': None},
 'x3': {'x': 3, 'x_type': 'int', 'writer': None},
 'x4': {'x': 4, 'x_type': 'int', 'writer': None},
 'x5': {'x': 5, 'x_type': 'int', 'writer': None},
 'x6': {'x': 6, 'x_type': 'int', 'writer': None},
 'x7': {'x': 7, 'x_type': 'int', 'writer': None},
 'x8': {'x': 8, 'x_type': 'int', 'writer': None},
 'x9': {'x': 9, 'x_type': 'int', 'writer': None},
 'x10': {'x': 10, 'x_type': 'int', 'writer': None},
 'x11': {'x': 11, 'x_type': 'int', 'writer': None},
 'x12': {'x': 12, 'x_type': 'int', 'writer': None},
 'x13': {'x': 13, 'x_type': 'int', 'writer': None},
 'x14': {'x': 14, 'x_type': 'int', 'writer': None},
 'x15': {'x': 15, 'x_type': 'int', 'writer': None},
 'x16': {'x': 16, 'x_type': 'int', 'writer': None},
 'x17': {'x': 17, 'x_type': 'int', 'writer': None},
 'x18': {'x': 18, 'x_type': 'int', 'writer': None},
 'x19': {'x': 19, 'x_type': 'int

## Parser das instruções

In [2]:
import parser_inst

In [6]:
OPCODES = {
    'fld': 0,
    'fsd': 1,
    'fadd': 2,
    'fsub': 3,
    'fmul': 4,
    'fdiv': 5
}

# Define register prefix constants
REG_PREFIXES = {
    'x': 'int',
    'f': 'float'
}

In [27]:
instruction_status =[]
with open("tests/ex.s", "r") as file:
    for i, line in enumerate(file):
        op = OPCODES[line.split()[0]]
        if (op == 0 or op == 1):
            rd = line.split()[1].replace(",", "")
            rs1 = line.split()[2].split("(")[1].replace(")", "")
            rs2 = None
            imm = line.split()[2].split("(")[0]
        else:
            rd = line.split()[1].replace(",", "")
            rs1 = line.split()[2].replace(",", "")
            rs2 = line.split()[3].replace(",", "")
            imm = None          
        instruction_status.append({ 
            "inst": line.replace("\n", ""),
            "opcode": op,
            "rd": rd,
            "rs1": rs1,
            "rs2": rs2,
            "imm": imm,
            "issue": None,
            "read_operands": None,
            "execution_complete": None,
            "write_result": None,
            "issue": None, 
            "read": None, 
            "exec": None, 
            "write": None

        })


In [29]:
for i, inst in enumerate(instruction_status):
    # print(i, inst)
    print(inst["inst"])


fld f1, 0(x1)
fsd  f1, 50(x11)
fadd f1, f2, f3
fsub f6, f3, f4
fmul f4, f8, f9
fdiv f2, f4, f5


In [ ]:
inst_status = []
instructions = parser_inst.parse_instructions("tests/ex.s")
for i, inst in enumerate(instructions):
    inst_status.append({f"instr": inst, "issue": None, "read": None, "exec": None, "write": None})
for i in inst_status:
    print(i)

In [3]:
import parser_inst
inst_status = []
instructions = parser_inst.parse_instructions("tests/ex.s")
for i, inst in enumerate(instructions):
    inst_status.append({f"instr": inst, "issue": None, "read": None, "exec": None, "write": None})
for i in inst_status:
    print(i)

{'instr': {'opcode': 0, 'rs1': 1, 'rs1_type': 'int', 'rs2': 0, 'rs2_type': None, 'rd': 1, 'rd_type': 'float', 'imm': 0}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 0, 'rs1': 1, 'rs1_type': 'int', 'rs2': 0, 'rs2_type': None, 'rd': 5, 'rd_type': 'float', 'imm': 0}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 5, 'rs1': 4, 'rs1_type': 'float', 'rs2': 5, 'rs2_type': 'float', 'rd': 2, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 4, 'rs1': 8, 'rs1_type': 'float', 'rs2': 9, 'rs2_type': 'float', 'rd': 4, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 2, 'rs1': 2, 'rs1_type': 'float', 'rs2': 3, 'rs2_type': 'float', 'rd': 1, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 1, 'rs1': 2, 'rs1_type': 'int', 'rs2': 4, 'rs2_type': 'float

## Parser das functionals units

In [4]:
fus_configs = {}
with open("tests/uf_config2.in", 'r') as f:
    linhas = f.readlines()
    for linha in linhas:
        partes = linha.strip().split()
        fus_configs[partes[0]] = {'qtd': partes[1], 'cycles': partes[2]}
fus_configs

{'int': {'qtd': '2', 'cycles': '1'}, 'div': {'qtd': '1', 'cycles': '10'}}

In [5]:
fus_status = {}
for uf_type, config in fus_configs.items():
    if uf_type == 'int':
        op = 0
    elif uf_type == 'mult':
        op = 1
    elif uf_type == 'add':
        op = 2
    elif uf_type == 'div':
        op = 3
    for c in range(int(config['qtd'])):
        fus_status[f'{uf_type}{c+1}'] = {
            'Opcode': op, 
            'Busy': False, 
            'Op': None, 
            'Fi': None,
            'Fj': None, 
            'Fk': None, 
            'Qj': None, 
            'Qk': None,
            'Rj': False, 
            'Rk': False, 
            'Cycles_left': int(config['cycles']), 
            'Cycles': int(config['cycles'])
        }
        

In [6]:
fus_status

{'int1': {'Opcode': 0,
  'Busy': False,
  'Op': None,
  'Fi': None,
  'Fj': None,
  'Fk': None,
  'Qj': None,
  'Qk': None,
  'Rj': False,
  'Rk': False,
  'Cycles_left': 1,
  'Cycles': 1},
 'int2': {'Opcode': 0,
  'Busy': False,
  'Op': None,
  'Fi': None,
  'Fj': None,
  'Fk': None,
  'Qj': None,
  'Qk': None,
  'Rj': False,
  'Rk': False,
  'Cycles_left': 1,
  'Cycles': 1},
 'div1': {'Opcode': 3,
  'Busy': False,
  'Op': None,
  'Fi': None,
  'Fj': None,
  'Fk': None,
  'Qj': None,
  'Qk': None,
  'Rj': False,
  'Rk': False,
  'Cycles_left': 10,
  'Cycles': 10}}

## Tabela Resultado

In [7]:
stages = {
    "Header": ["Instruction/Cycle", "Issue", "Read", "Execute", "Write"],
    "Instruction/Cicle": [None]*len(inst_status),
    "Issue": [None]*len(inst_status),
    "Read": [None]*len(inst_status),
    "Execute": [None]*len(inst_status),
    "Write": [None]*len(inst_status)
}

## ISSUE

Verificar se a unidade funcional requerida está livre -- sem hazards estruturais
    
    Functional unit status -- verificada aqui

Verificar se nenhuma outra instrução vai escrever no rd -- sem WAW hazards
    
    Register Status -- verificado aqui

In [8]:
pc = 0
inst_info = inst_status[pc]['instr']
inst_info

{'opcode': 0,
 'rs1': 1,
 'rs1_type': 'int',
 'rs2': 0,
 'rs2_type': None,
 'rd': 1,
 'rd_type': 'float',
 'imm': 0}

In [9]:
instr = inst_status[0]['instr']
opcode = instr['opcode']
rs1 = instr['rs1']
rs1_type = instr['rs1_type']
rs2 = instr['rs2']
rs2_type = instr['rs2_type']
rd = instr['rd']
rd_type = instr['rd_type']

prefix_rd = "x" if rd_type == "int" else "f"
key_rd = f"{prefix_rd}{rd}"

prefix_rs1 = "x" if rs1_type == "int" else "f"
key_rs1 = f"{prefix_rs1}{rs1}"

prefix_rs2 = "x" if rs2_type == "int" else "f"
key_rs2 = f"{prefix_rs2}{rs2}"

In [10]:
idx = 0  # ou o índice correto da instrução que está sendo processada
opcodes = {0, 1, 2, 3, 4, 5} 

for fu_name, fu in fus_status.items():
    if fu['Opcode'] != opcode or fu['Busy']:
        continue
    if  register_status[key_rd]['writer'] is not None:
        continue
    fu['Busy'] = True
    fu['Op'] = opcode
    fu['Fi'] = key_rd if opcode in opcodes else None
    fu['Fj'] = key_rs1
    fu['Fk'] = key_rs2
    fu['Qj'] = register_status[key_rs1]['writer']
    fu['Qk'] = register_status[key_rs2]['writer']
    fu['Rj'] = fu['Qj'] is None
    fu['Rk'] = fu['Qk'] is None
    fu['CyclesLeft'] = fu['Cycles']
    
    if opcode in opcodes:
        register_status[key_rd]['writer'] = fu_name

    inst_status[idx]['issue'] = 1
    stages['Instruction/Cicle'][idx] = inst_status[idx]['instr']
    stages['Issue'][idx] = inst_status[idx]['issue']

In [11]:
fus_status

{'int1': {'Opcode': 0,
  'Busy': True,
  'Op': 0,
  'Fi': 'f1',
  'Fj': 'x1',
  'Fk': 'f0',
  'Qj': None,
  'Qk': None,
  'Rj': True,
  'Rk': True,
  'Cycles_left': 1,
  'Cycles': 1,
  'CyclesLeft': 1},
 'int2': {'Opcode': 0,
  'Busy': False,
  'Op': None,
  'Fi': None,
  'Fj': None,
  'Fk': None,
  'Qj': None,
  'Qk': None,
  'Rj': False,
  'Rk': False,
  'Cycles_left': 1,
  'Cycles': 1},
 'div1': {'Opcode': 3,
  'Busy': False,
  'Op': None,
  'Fi': None,
  'Fj': None,
  'Fk': None,
  'Qj': None,
  'Qk': None,
  'Rj': False,
  'Rk': False,
  'Cycles_left': 10,
  'Cycles': 10}}

In [12]:
register_status[key_rd]

{'f': 1, 'f_type': 'float', 'writer': 'int1'}

fld f1, 0(x1)

Op=0

Fi: f1
Fj: x1
Fk: -
Qj: -
Qk: -
Rj: Yes
Rk: Yes

## Read Operands

In [13]:
for i in inst_status:
    print(i)

{'instr': {'opcode': 0, 'rs1': 1, 'rs1_type': 'int', 'rs2': 0, 'rs2_type': None, 'rd': 1, 'rd_type': 'float', 'imm': 0}, 'issue': 1, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 0, 'rs1': 1, 'rs1_type': 'int', 'rs2': 0, 'rs2_type': None, 'rd': 5, 'rd_type': 'float', 'imm': 0}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 5, 'rs1': 4, 'rs1_type': 'float', 'rs2': 5, 'rs2_type': 'float', 'rd': 2, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 4, 'rs1': 8, 'rs1_type': 'float', 'rs2': 9, 'rs2_type': 'float', 'rd': 4, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 2, 'rs1': 2, 'rs1_type': 'float', 'rs2': 3, 'rs2_type': 'float', 'rd': 1, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 1, 'rs1': 2, 'rs1_type': 'int', 'rs2': 4, 'rs2_type': 'float', 

In [14]:
instr

{'opcode': 0,
 'rs1': 1,
 'rs1_type': 'int',
 'rs2': 0,
 'rs2_type': None,
 'rd': 1,
 'rd_type': 'float',
 'imm': 0}

In [ ]:
if register_status[key_rd]['writer'] is not None:
    register_status[key_rd]['']

'int1'

In [ ]:
instr = inst_status[0]['instr']


stages['Issue'][idx] = inst_status[idx]['issue']


## Scoreboard

### Montando a instrução novamente

In [ ]:
def any_fu_busy(fus):
    for fu_name, fu in fus.items():
        if fu['Busy']:
            return True
    return False

In [ ]:
cycle = 0
pc = 0

while pc < len(inst_status) or any_fu_busy(fus_status):
    cycle += 1
    print
    # # Tenta emitir a próxima instrução
    # if pc < len(inst_status):
    #     if issue_instruction(pc, cycle, inst_status, fus, register_status, inst_status):
    #         pc += 1 # Avança o PC somente se a emissão for bem-sucedida

In [ ]:
for idx, i in enumerate(stages['Instruction/Cicle']):
    if i is not None:
        if i['opcode'] == 0:
            rd = f"f{i['rd']}" if i['rd_type'] == 'float' else f"x{i['rd']}"
            rs1 = f"f{i['rs1']}" if i['rs1_type'] == 'float' else f"x{i['rs1']}"
            imm = i.get('imm', 0)
            i_mont = f"fld {rd}, {imm}({rs1})"
            stages['Instruction/Cicle'][idx] = i_mont 


In [ ]:
df_stages = pd.DataFrame({
    "Instruction/Cycle": stages["Instruction/Cicle"],
    "Issue": stages["Issue"],
    "Read": stages["Read"],
    "Execute": stages["Execute"],
    "Write": stages["Write"]
})

# df_stages["Issue"] = df_stages["Issue"].astype("Int64")

# df_stages.style.hide(axis="index")
print(df_stages.to_string(index=False))